In [271]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from datetime import date

In [217]:
train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")

In [203]:
train.head(10)

,Deal_title,Lead_name,Industry,Deal_value,Weighted_amount,Date_of_creation,Pitch,Contact_no,Lead_revenue,Fund_category,...,Designation,Lead_POC_email,Hiring_candidate_role,Lead_source,Level_of_meeting,Last_lead_update,Internal_POC,Resource,Internal_rating,Success_probability
0,TitleM5DZY,"Davis, Perkins and Bishop Inc",Restaurants,320506$,2067263.7$,2020-03-29,Product_2,607.447.7883,50 - 100 Million,Category 2,...,Executive Vice President,charlenewerner@davis.com,Community pharmacist,Website,Level 3,No track,"Davis,Sharrice A",NaN,3,73.6
1,TitleKIW18,Bender PLC LLC,Construction Services,39488$,240876.8$,2019-07-10,Product_2,892-938-9493,500 Million - 1 Billion,Category 4,...,Chairman/CEO/President,terrylogan@bender.com,Recruitment consultant,Others,Level 1,Did not hear back after Level 1,"Brown,Maxine A",No,5,58.9
2,TitleFXSDN,Carter-Henry and Sons,Hospitals/Clinics,359392$,2407926.4$,2019-07-27,Product_1,538.748.2271,500 Million - 1 Billion,Category 4,...,SVP/General Counsel,arielhamilton@carterhenry.com,Health service manager,Marketing Event,Level 1,?,"Georgakopoulos,Vasilios T",No,4,68.8
3,TitlePSK4Y,Garcia Ltd Ltd,Real Estate,76774$,468321.4$,2021-01-30,Product_2,(692)052-1389x75188,500 Million - 1 Billion,Category 3,...,CEO/Co-Founder/Chairman,erinwilson@garcia.com,"Therapist, speech and language",Contact Email,Level 2,Did not hear back after Level 1,"Brown,Maxine A",We have all the requirements,1,64.5
4,Title904GV,Lee and Sons PLC,Financial Services,483896$,NaN,2019-05-22,Product_2,001-878-814-6134x015,50 - 100 Million,Category 3,...,Executive Vice President,mr.christopher@lee.com,Media planner,Website,Level 2,Up-to-date,"Thomas,Lori E",No,4,62.4
5,Title00VOR,Chavez Ltd Inc,Banks,418674$,2637646.2$,2019-06-30,Product_1,(418)259-9934x952,50 - 100 Million,Category 2,...,CEO/Co-Founder/Chairman,crystalchavez@chavez.com,Microbiologist,Marketing Event,Level 3,2 days back,"Featherstone,Adrian R",Deliverable,3,66.3
6,TitleOZQRY,Williamson LLC and Sons,Banks,384356$,2709709.8$,2019-11-20,Product_2,(881)077-4692,500 Million - 1 Billion,Category 1,...,CEO/Co-Founder/Chairman,saradixon@williamson.com,Cartographer,Contact Email,Level 3,More than 2 weeks,"Booker,David L",NaN,4,73.4
7,TitleV05WV,"Livingston, York and Adams Group",Architecture/Engineering,245205$,1642873.5$,2020-12-22,Product_2,953.762.9149,500 Million - 1 Billion,Category 1,...,Vice President / GM (04-present) : VP Sales an...,christianstanley@livingston.com,"Engineer, maintenance (IT)",Others,Level 2,5 days back,"Cashin,Marc C",No,5,74.1
8,TitleC2RNN,Powers Ltd Inc,Education/Training,343280$,1991024.0$,2019-01-08,Product_1,(566)194-6345,100 - 500 Million,Category 3,...,Chairman/Chief Innovation Officer,josephthompson@powers.com,Product/process development scientist,Contact Email,Level 1,No track,"Van Arter,Derrick",We have all the requirements,2,61.4
9,TitleDZT40,Shea Group PLC,REIT,293611$,1791027.1$,2020-07-04,Product_1,(292)819-5746,500 Million - 1 Billion,Category 3,...,CEO,williamgrimes@shea.com,Engineering geologist,Contact Email,Level 1,more than a month,"Hanyok,John J",Deliverable,4,66.0


In [204]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7007 entries, 0 to 7006
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Deal_title             7007 non-null   object 
 1   Lead_name              7007 non-null   object 
 2   Industry               7006 non-null   object 
 3   Deal_value             6956 non-null   object 
 4   Weighted_amount        6482 non-null   object 
 5   Date_of_creation       7007 non-null   object 
 6   Pitch                  7007 non-null   object 
 7   Contact_no             7007 non-null   object 
 8   Lead_revenue           7007 non-null   object 
 9   Fund_category          7007 non-null   object 
 10  Geography              6035 non-null   object 
 11  Location               6996 non-null   object 
 12  POC_name               6999 non-null   object 
 13  Designation            7007 non-null   object 
 14  Lead_POC_email         7007 non-null   object 
 15  Hiri

# Clean train data

In [218]:
train = train.drop(['Deal_title', 'Lead_name', 'Contact_no', 'Lead_POC_email'], axis = 1)

In [219]:
train['Date_of_creation'] = pd.to_datetime(train['Date_of_creation'])
train['days_since_created'] = train['Date_of_creation'].apply(lambda x:(pd.Timestamp('today') - x).days)
train.drop(['Date_of_creation'], inplace=True, axis=1)

In [221]:
train.Deal_value = train.Deal_value.apply(lambda x : str(x)[:-1] if str(x)[-1] == '$' else x)
train.Weighted_amount = train.Weighted_amount.apply(lambda x : str(x)[:-1] if str(x)[-1] == '$' else x)

In [222]:
train['min_revenue'] = train.Lead_revenue.apply(lambda x: x.split('-')[0])
train['max_revenue'] = train.Lead_revenue.apply(lambda x: x.split('-')[1])
train = train.drop(['Lead_revenue'], axis = 1)

In [223]:
train.min_revenue = train.min_revenue.apply(lambda x: '500' if x == '500 Million ' else x)
train.max_revenue = train.max_revenue.apply(lambda x: '100' if x == ' 100 Million' else ('500' if x == ' 500 Million' else '1000'))

In [224]:
train["Deal_value"] = train.Deal_value.astype(float)
train["Weighted_amount"] = train.Weighted_amount.astype(float)
train["min_revenue"] = train.min_revenue.astype(float)
train["max_revenue"] = train.max_revenue.astype(float)

In [225]:
train['Weighted_amount'] = train['Weighted_amount'].fillna((train['Weighted_amount'].mean()))
train['Deal_value'] = train['Deal_value'].fillna((train['Deal_value'].mean()))

In [227]:
train['Geography'] = train['Geography'].fillna(method = 'ffill')
train['Resource'] = train['Resource'].fillna(method = 'bfill')

In [233]:
def fill(train, s):
    n = 7007
    x = 'No'
    for i in range(n):
        if train[s][i] == '?':
            train[s][i] = x
        x = train[s][i]
    return train
train = fill(train, 'Last_lead_update')

<ipython-input-232-338bb2f42772>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[s][i] = x


In [236]:
train.Success_probability = train.Success_probability.apply(lambda x: -x if x < 0 else x)

In [237]:
train.describe()

,Deal_value,Weighted_amount,Internal_rating,Success_probability,days_since_created,min_revenue,max_revenue
count,7007.000000,7.007000e+03,7007.000000,7007.000000,7007.000000,7007.000000,7007.000000
mean,249656.025446,1.569884e+06,3.009562,64.845034,440.625089,219.430569,539.517625
std,144231.008181,8.863772e+05,1.418666,17.566890,224.690222,202.070367,367.838423
min,1551.000000,8.708000e+03,1.000000,5.000000,52.000000,50.000000,100.000000
25%,123293.500000,8.264284e+05,2.000000,60.600000,247.500000,50.000000,100.000000
50%,248796.000000,1.569884e+06,3.000000,65.300000,440.000000,100.000000,500.000000
75%,375706.500000,2.293031e+06,4.000000,69.600000,635.000000,500.000000,1000.000000
max,500000.000000,3.601416e+06,5.000000,107.340000,828.000000,500.000000,1000.000000


In [266]:
train = pd.get_dummies(train)

In [267]:
train.head()

,Deal_value,Weighted_amount,Internal_rating,Success_probability,days_since_created,min_revenue,max_revenue,Industry_Accounting,Industry_Advertising/PR,Industry_Aerospace/Defense,...,"Internal_POC_Ullrich,Rose Anne","Internal_POC_Van Arter,Derrick","Internal_POC_Vickers Jr.,Henry J","Internal_POC_Young,Valerie K",Resource_Cannot deliver,Resource_Deliverable,Resource_No,Resource_Not enough,Resource_We have all the requirements,Resource_Yes
0,320506.0,2.067264e+06,3,73.6,375,50.0,100.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,39488.0,2.408768e+05,5,58.9,638,500.0,1000.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,359392.0,2.407926e+06,4,68.8,621,500.0,1000.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,76774.0,4.683214e+05,1,64.5,68,500.0,1000.0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,483896.0,1.569884e+06,4,62.4,687,50.0,100.0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


# clean test data

In [238]:
Deal_title = test.Deal_title
test = test.drop(['Deal_title', 'Lead_name', 'Contact_no', 'Lead_POC_email'], axis = 1)

In [239]:
test['Date_of_creation'] = pd.to_datetime(test['Date_of_creation'])
test['days_since_created'] = test['Date_of_creation'].apply(lambda x:(pd.Timestamp('today') - x).days)
test.drop(['Date_of_creation'], inplace=True, axis=1)

In [240]:
test.Deal_value = test.Deal_value.apply(lambda x : str(x)[:-1] if str(x)[-1] == '$' else x)
test.Weighted_amount = test.Weighted_amount.apply(lambda x : str(x)[:-1] if str(x)[-1] == '$' else x)

In [241]:
test['min_revenue'] = test.Lead_revenue.apply(lambda x: x.split('-')[0])
test['max_revenue'] = test.Lead_revenue.apply(lambda x: x.split('-')[1])
test = test.drop(['Lead_revenue'], axis = 1)

In [242]:
test.min_revenue = test.min_revenue.apply(lambda x: '500' if x == '500 Million ' else x)
test.max_revenue = test.max_revenue.apply(lambda x: '100' if x == ' 100 Million' else ('500' if x == ' 500 Million' else '1000'))

In [244]:
test["Deal_value"] = test.Deal_value.astype(float)
test["Weighted_amount"] = test.Weighted_amount.astype(float)
test["min_revenue"] = test.min_revenue.astype(float)
test["max_revenue"] = test.max_revenue.astype(float)

In [245]:
test['Weighted_amount'] = test['Weighted_amount'].fillna((test['Weighted_amount'].mean()))
test['Deal_value'] = test['Deal_value'].fillna((test['Deal_value'].mean()))

In [246]:
test['Geography'] = test['Geography'].fillna(method = 'ffill')
test['Resource'] = test['Resource'].fillna(method = 'bfill')

In [249]:
def fill(test, s):
    n = 2093
    x = 'No'
    for i in range(n):
        if test[s][i] == '?':
            test[s][i] = x
        x = test[s][i]
    return test
test = fill(test, 'Last_lead_update')

<ipython-input-249-d90b6ffb99be>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test[s][i] = x


In [264]:
test.Internal_rating = test.Internal_rating.apply(lambda x: -x if x < 0 else x)

In [265]:
test.describe()

,Deal_value,Weighted_amount,Internal_rating,days_since_created,min_revenue,max_revenue
count,2093.000000,2.093000e+03,2093.000000,2093.000000,2093.000000,2093.000000
mean,248052.138889,1.556917e+06,3.189221,441.614907,213.186813,525.704730
std,141862.423966,8.873994e+05,4.478122,223.069168,200.502528,368.447914
min,2025.000000,1.316250e+04,1.000000,52.000000,50.000000,100.000000
25%,126390.000000,8.049900e+05,2.000000,251.000000,50.000000,100.000000
50%,250650.000000,1.556917e+06,3.000000,441.000000,100.000000,500.000000
75%,372023.000000,2.311754e+06,4.000000,636.000000,500.000000,1000.000000
max,499392.000000,3.409163e+06,82.340000,828.000000,500.000000,1000.000000


In [268]:
test = pd.get_dummies(test)

In [269]:
test.head()

,Deal_value,Weighted_amount,Internal_rating,days_since_created,min_revenue,max_revenue,Industry_Accounting,Industry_Advertising/PR,Industry_Aerospace/Defense,Industry_Airlines/Air Couriers,...,"Internal_POC_Ullrich,Rose Anne","Internal_POC_Van Arter,Derrick","Internal_POC_Vickers Jr.,Henry J","Internal_POC_Young,Valerie K",Resource_Cannot deliver,Resource_Deliverable,Resource_No,Resource_Not enough,Resource_We have all the requirements,Resource_Yes
0,200988.0,1.556917e+06,1.0,358,100.0,500.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,409961.0,2.541758e+06,1.0,75,100.0,500.0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,434433.0,3.041031e+06,5.0,263,100.0,500.0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,218952.0,1.521716e+06,5.0,406,100.0,500.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,392835.0,2.455219e+06,2.0,165,500.0,1000.0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [272]:
X_train, X_val, y_train, y_val = train_test_split(train.drop(['Success_probability'], axis = 1), train.Success_probability, test_size=0.2, random_state=42)